# 将检索模块和文本聚类模块进行封装

In [1]:
import time
import random
import os
import re
import pandas as pd
import numpy as np
from datetime import datetime
from elasticsearch import Elasticsearch

# ES检索模块重构为一个对象

In [67]:
class esPaperRetrieval():
    '''
    根据论文检索需求进行功能的微调
    '''
    _instance = None
    _first_init = True
    def __new__(cls, *args, **kw):
        if not cls._instance:
            cls._instance = super(esPaperRetrieval, cls).__new__(cls)  
        return cls._instance
    
    def __init__(self, host, port):
        '''
        使用ES进行论文检索 指定host、port以及专利index之后进行检索
        '''
        super(esPaperRetrieval, self).__init__()
        self.es = Elasticsearch(hosts=host, port=port, timeout=30, max_retries=10, retry_on_timeout=True)
        self.indexName = 'paper-detail-index'

    def do_search(self, titleQuery, kwordQuery, summaryQuery ,pubQuery,fromDate, toDate, volume):
        '''
        do_search方法执行具体检索过程
        titleQuery 为对应标题检索词
        kwordQuery 为对应关键字检索词
        summaryQuery 为对应摘要检索词
        上述三个词为or模式 可以出现 可以不出现
        pubQuery 为对应文摘关键词 这个关键词必定匹配
        fromDate 为检索字段起始日期 toDate为终结日期 日期格式 yyyy-mm-dd
        volume为每次检索返回的数目
        '''
        queryBody = {
          "query": {
            "bool": {
              "should": [
                {
                  "term": {
                    "P_Title": titleQuery
                  }
                },
                {
                  "term": {
                    "P_Keyword": kwordQuery
                  }
                },
                {
                  "term": {
                    "P_Summary": summaryQuery
                  }
                },
                  {
                  "term": {
                    "P_Publication.keyword": {"value":pubQuery}
                  }
                  }
              ],
              "filter": [
                {
                  "range": {
                    "P_year": {
                      "gt": fromDate,
                      "lt": toDate
                    }
                  }
                },
              ]
            }
          },
            "from": 0,
            "size": volume,
            "sort": [],
            "aggs": {}
        }
#         print(queryBody)
        result = self.es.search(index=self.indexName, body=queryBody)
        return result

    def format_search(self, result):
        '''
        format_search方法对检索结果进行格式化 构建符合要求的字段进行返回
        输入result为检索结果 提取其中的检索结果进行后处理
        使用ES检索后得到的结果中result['hits']['hits']为数组格式数据
        其中每一个元素为一个dict 对应部分字段
        '''
        docs = result['hits']['hits']
        docs = [i['_source'] for i in docs]
        targetKeyList = 'P_ID, P_Title, P_Author, P_Publication, P_Organ, P_year, P_Keyword, \
    P_Summary, P_Keyword_seg, P_Title_seg,\
    P_Summary_seg, P_URL, P_Fields, P_Fields_two,P_References, P_Pagecount, P_Page, P_Language,\
    P_Download_num, P_Citation_num,P_Vector,P_Volume, P_Issue,P_Issn,P_Isbn, P_Doi,\
    P_Red1, P_Red2, P_Red3, P_Red4, P_Red5'
        targetKeyList = [i.strip() for i in targetKeyList.split(',')]
        dict_filter_by_keys = lambda d: {k: d[k] for k in targetKeyList}
        dict_filter_text = lambda d: {k if not k == 'text' else 'claim_text': d[k] for k in d}
        dict_filter_id = lambda d: {k if not k == '_id' else 'id': d[k] for k in d}
        docs = (dict_filter_by_keys(doc) for doc in docs)
        docs = [dict_filter_text(doc) for doc in docs]
        return docs
    def Retrieval(self, titleQuery, kwordQuery, summaryQuery ,pubQuery,fromDate, toDate, volume):
        result = self.do_search(titleQuery, kwordQuery, summaryQuery ,pubQuery,fromDate, toDate, volume)
        docs = self.format_search(result)
        return docs

In [68]:
es4Paper = esPaperRetrieval(host='10.8.128.205',port=49200,)

In [69]:
result= es4Paper.Retrieval(titleQuery='医',kwordQuery='医',summaryQuery='医=',pubQuery='',
              fromDate='2005-01-01',toDate='2018-01-01',volume=10)

In [70]:
result[0]

{'P_ID': 1129492,
 'P_Title': '医德医风与医院履责对医生履责的影响研究',
 'P_Author': '蒋重阳;周萍;白飞;张瑜;朱洪波;魏雪峰;',
 'P_Publication': '中国医院管理',
 'P_Organ': '复旦大学公共卫生学院·卫生部卫生技术评估重点实验室(复旦大学)·健康风险预警治理协同创新中心;国家卫生计生委医疗管理服务指导中心;湖北省卫生和计划生育委员会;甘肃省卫生和计划生育委员会;',
 'P_year': '2017-01-01',
 'P_Keyword': '医德医风;;医院履责;;医生履责',
 'P_Summary': '目的探讨医院医德医风和医生感知的医院责任履行状况对医生自身责任履行的影响。方法运用自行设计的调查问卷对我国东、中、西3省、直辖市9个地区的128所公立医院医生进行调查。结果层次回归分析显示,医德医风和医院履责对医生自身履责解释力均显著。在控制了个体变量与医院地区级别变量之后,医德医风对医生交易、发展、关系与理念责任的履行均具有显著正向影响(P<0.001),而医院履责对医生履责的影响有不同的对应性。结论医院医德医风对医生自身责任的履行有积极促进作用。医院4类责任履行状况与医生4类责任履行有紧密关联,但对应性略有不同。',
 'P_Keyword_seg': '医德医风 医院履责 医生履责',
 'P_Title_seg': '医德医风 与 医院履责 对 医生履责 的 影响研究',
 'P_Summary_seg': '目的 探讨 医院 医德医风 和 医生 感知 的 医院 责任履行 状况 对 医生 自身 责任履行 的 影响 。 方法 运用 自行设计 的 调查问卷 对 我国 东 、 中 、 西 3 省 、 直辖市 9 个 地区 的 12 8 所 公立医院 医生 进行 调查 。 结果 层次回归分析 显示 , 医德医风 和 医院履责 对 医生 自身 履责 解释 力均 显著 。 在 控制 了 个体变量 与 医院 地区 级别 变量 之后 , 医德医风 对 医生 交易 、 发展 、 关系 与 理念 责任 的 履行 均 具有 显著 正向影响 ( P < 0.001 ) , 而 医院履责 对 医生履责 的 影响 有 不同 的 对应性 。 结论 医院 医德医风 对 医生 自身 责任 的 履行 

In [56]:
def post_process(result_list):
    target_keys = ['P_ID','P_Title','P_Author','P_Publication','P_year']
    result_list = [{k:i[k] for k in target_keys} for i in result_list]
    return result_list

# 客户端对于后端数据接口的访问

In [4]:
import requests

body = {'searchQuery':'机器人',
        'fromDate':'2000-06-02',
        'toDate':'2016-01-01',
        'volume':10000}

baseUrl = 'http://10.8.128.205:29280/Lawbda/dataWare/1.0.0/patent/search'

docs = requests.get(baseUrl,params=body)

# 通过swagger 获取数据

In [5]:
import requests

In [6]:
base_url = 'http://10.8.128.205:29280/Lawbda/dataWare/1.0.0/paper/search'

In [7]:
payload = {
    'titleQuery':'医',
    'kwdQuery':'医',
    'smryQuery':'医',
    'pubQuery':'世界中医药',
    'fromDate':'2001-01-01',
    'toDate':'2018-01-01',
    'dateOrder':'asc',
    'volume':100
}

In [8]:
ppp = requests.get(base_url,params=payload)

In [9]:
ppp.json()

[{'P_Author': '周冰;南继红;陈荠;崔永强;软洪琤;',
  'P_ID': 1585620,
  'P_Publication': '世界中医药',
  'P_Title': '西班牙中医服务与医疗保险覆盖现状',
  'P_year': '2017-01-01'},
 {'P_Author': '周冰;南继红;陈荠;崔永强;软洪琤;',
  'P_ID': 5007,
  'P_Publication': '世界中医药',
  'P_Title': '西班牙中医服务与医疗保险覆盖现状',
  'P_year': '2017-01-01'},
 {'P_Author': '郭志武;李惠林;李顺民;',
  'P_ID': 1585332,
  'P_Publication': '世界中医药',
  'P_Title': '中医医案标准体系构建探析',
  'P_year': '2015-01-01'},
 {'P_Author': '郭志武;李惠林;李顺民;',
  'P_ID': 4719,
  'P_Publication': '世界中医药',
  'P_Title': '中医医案标准体系构建探析',
  'P_year': '2015-01-01'},
 {'P_Author': '赵维婷;',
  'P_ID': 1583703,
  'P_Publication': '世界中医药',
  'P_Title': '瑞士将中医提升到与西医平等地位',
  'P_year': '2016-01-01'},
 {'P_Author': '赵维婷;',
  'P_ID': 3090,
  'P_Publication': '世界中医药',
  'P_Title': '瑞士将中医提升到与西医平等地位',
  'P_year': '2016-01-01'},
 {'P_Author': '徐晓婷;张海明;',
  'P_ID': 1584772,
  'P_Publication': '世界中医药',
  'P_Title': '由医疗器械国际标准化特点试论中医医疗器械国际标准化发展',
  'P_year': '2017-01-01'},
 {'P_Author': '徐晓婷;张海明;',
  'P_ID': 4159,
  'P_Publicatio